In [3]:
import numpy as np
import sympy as sp
sp.init_printing()

In [6]:
def calculate_chrst_sym_4d(metric, inv_metric, derivs = [t0,t1,t2,t3]):
    #Calculate the christoffel symbols for a given metric
    
    #used to take deriv by correct variable
    deriv_list = derivs

    #empty array to put our christoffel symbols
    chrst = np.zeros((4, 4, 4), dtype = 'object') 
    
    for k in range(4):
        for i in range(4):
            for j in range(4):
                pre_sum = 0
                for m in range(4):
                    pre_sum += .5 * inv_metric[k][m] * (
                      sp.diff(metric[m][i], deriv_list[j]) + sp.diff(metric[m][j], deriv_list[i]) - sp.diff(metric[i][j], deriv_list[m])
                    )
                chrst[i,j,k] = pre_sum

    return chrst

In [7]:
from sympy.stats import E

def sym_cub_tensor(der):
    #symmetric cubic tensor given n derivatives
    
    n = len(der)    
    return_mat = np.zeros((n,n,n), dtype = 'object')
    
    for k in range(n):
        for i in range(n):
            for j in range(n):
                return_mat[i,j,k] = sp.simplify(E(der[k] * der[i] * der[j]))
    
    return return_mat

In [8]:
def find_alpha_chrst(chrst, tijks, alpha):
    #finds alpha christoffel symbols
    return chrst + (alpha / 2) * tijks

In [44]:
def alpha_curvature_tensor(alpha_chrsts):
    #finds standard curvature
    
    varis = [t0,t1,t2,t3]
    
    alpha_curve_tensor_arr = np.zeros((4,4,4,4), dtype = 'object')
    
    for l in range(4):
        for i in range(4):
            for j in range(4):
                for k in range(4):
                    pre_sum = sp.diff(alpha_chrsts[i][k][l], varis[j]) - sp.diff(alpha_chrsts[j][k][l], varis[i])
                    for p in range(4):
                        pre_sum += alpha_chrsts[i][k][p] * alpha_chrsts[j][p][l] - alpha_chrsts[j][k][p] * alpha_chrsts[i][p][l]
                    alpha_curve_tensor_arr[i][j][k][l] = sp.simplify(pre_sum)
    
    return alpha_curve_tensor_arr

In [41]:
def curvature_components(curv_tens, metric):
    #finds curvature components
    
    n = np.shape(curv_tens)[0]
    
    return_arr = np.zeros((n,n,n,n), dtype = 'object')
    
    for i in range(4):
        for j in range(4):
            for k in range(4):
                for l in range(4):
                    pre_sum = 0
                    for s in range(4):
                        pre_sum += curv_tens[i,j,k,l] * metric[l,s]
                    return_arr[i,j,k,l] = pre_sum
        
    return return_arr

In [61]:
def curvature_numerator(comps):
    #finds the numerator of the value of the curvature
    
    #creating vector terms
    u0, u1, u2, u3 = sp.symbols('u1 u2 u3 u4')
    
    v0, v1, v2, v3 = sp.symbols('v1:5')
    
    u_list = [u0, u1, u2, u3]
    v_list = [v0, v1, v2, v3]
    
    pre_sum = 0
    
    for j in range(4):
        for i in range(4):
            for l in range(4):
                for k in range(4):
                    sporting = v_list[j] * u_list[i] * comps[i,j,k,l] * u_list[l] * v_list[k]
                    pre_sum += sporting
                    #error could be above, lk are swapped from curvature tensor to formula
    
    return pre_sum

In [14]:
#creating variables for use

t0 = sp.Symbol('t0', real=True)
t1 = sp.Symbol('t1', positive = True)
t2 = sp.Symbol('t2', real=True)
t3 = sp.Symbol('t3', positive = True)

from sympy.stats import Normal

x, y = Normal('x', t0, t1), Normal('y', t2, t3)

In [16]:
#S2 x S2 metric from bad paper

s22_met = np.zeros((4,4), dtype = 'object')
s22_met[0][0] = 1
s22_met[1][1] = sp.sin(t0) ** 2
s22_met[2][2] = 1
s22_met[3][3] = sp.sin(t2) ** 2

In [17]:
#S2 x S2 inverse metric from bad paper

s22_inv_met = np.zeros((4,4), dtype = 'object')
s22_inv_met[0][0] = 1
s22_inv_met[1][1] = sp.sin(t0) ** (-2)
s22_inv_met[2][2] = 1
s22_inv_met[3][3] = sp.sin(t2) ** (-2)

In [18]:
#H x H metric

h22_met = np.zeros((4,4), dtype = 'object')
h22_met[0][0] = 1
h22_met[1][1] = 1 / t1
h22_met[2][2] = 1
h22_met[3][3] = 1 / t3

In [19]:
#H x H inverse metric

h22_inv_met = np.zeros((4,4), dtype = 'object')
h22_inv_met[0][0] = 1
h22_inv_met[1][1] = t1
h22_inv_met[2][2] = 1
h22_inv_met[3][3] = t3

In [29]:
#We perform step 3
step_3_s22 = calculate_chrst_sym_4d(s22_met, s22_inv_met)
step_3_s22
# step_3_h22 = calculate_chrst_sym_4d(h22_met, h22_inv_met)

array([[[0, 0, 0, 0],
        [0, 1.0*cos(t0)/sin(t0), 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 1.0*cos(t0)/sin(t0), 0, 0],
        [-1.0*sin(t0)*cos(t0), 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 1.0*cos(t2)/sin(t2)]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 1.0*cos(t2)/sin(t2)],
        [0, 0, -1.0*sin(t2)*cos(t2), 0]]], dtype=object)

In [21]:
#For step 4 we first define the derivatives

der_1 = (x - t0) / (t1 ** 2)
der_2 = (((x - t0) ** 2) / (t1 ** 3)) - 1 / t1
#derivative take wrt variance
#der_2 = (((x - t0) ** 2) / (2 * (t1 ** 4))) - 1 /(2 * t1 ** 2)
der_3 = (y - t2) / (t3 ** 2)
der_4 = (((y - t2) ** 2) / (t3 ** 3)) - 1 / t3
#der_4 = (((y - t2) ** 2) / (2 * (t3 ** 4))) - 1 /(2 * t3 ** 2)

der_list = [der_1,der_2,der_3,der_4]

In [ ]:
#Step 4 calculation
#try to not run this cell multiple times
step_4_tensor = sym_cub_tensor(der_list)

In [ ]:
step_5_arr_pos1 = find_alpha_chrst(step_3_h22, step_4_tensor, 1)
step_5_arr_neg1 = find_alpha_chrst(step_3_h22, step_4_tensor, -1)

In [ ]:
step_6_arr_pos1 = alpha_curvature_tensor(step_5_arr_pos1)
step_6_arr_neg1 = alpha_curvature_tensor(step_5_arr_neg1)

In [38]:
step_7_arr_pos1 = curvature_components(step_6_arr_pos1, s22_met)

step_7_arr_pos1

NameError: name 'step_6_arr_pos1' is not defined

In [49]:
#doing alpha = 0 case
#we have up to step 3, and will not be doing 4/5

step_6_a0 = alpha_curvature_tensor(step_3_s22)

In [53]:
step_7_a0 = curvature_components(step_6_a0, s22_met)

In [65]:
step_8_a0 = curvature_numerator(step_7_a0)